The goal of this notebook is to tie the songs I know into the era that they are from. 

For now I am just focusing on non classical music, essentially 20th and 21st century.

In [1]:
# Refactored pipeline: artists by decade (+ excluded artists section)
# This cell is self-contained and does not rely on earlier cells.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Markdown, display

# ----------------------
# Config & helpers
# ----------------------
OUT_DIR = Path('obsidian_export')
OUT_DIR.mkdir(parents=True, exist_ok=True)
IMG_PATH = OUT_DIR / 'artists_by_decade.png'

# Filters to exclude "classical" content
FILTER_GENRE_TERMS = ['classical', 'choral', 'baroque']
FILTER_ALBUM_TERMS = ['Bach']


def clean_artist_col(series: pd.Series) -> pd.Series:
    """Return primary artist (first before comma), trimmed."""
    return series.astype(str).str.split(',').str[0].str.strip()


def infer_year_from(
    df: pd.DataFrame,
    candidates=('Album Date', 'Release Date'),
    text_cols=('Album', 'Song', 'Title', 'Track', 'Name'),
) -> pd.Series:
    """Infer a year from date columns first, then regex from text columns."""
    year = pd.Series(np.nan, index=df.index, dtype='float')

    for col in candidates:
        if col in df.columns:
            # Try parsing as datetime then take year
            y_dt = pd.to_datetime(df[col], errors='coerce', utc=True).dt.year
            year = year.fillna(y_dt)

            # Try numeric-as-year if provided directly
            y_num = pd.to_numeric(df[col], errors='coerce')
            y_num = y_num.where(y_num.between(1800, 2099))
            year = year.fillna(y_num)

    # Fallback regex search for a 4-digit plausible year in text fields
    regex = r'(?<!\d)(18|19|20)\d{2}(?!\d)'
    for col in text_cols:
        if col in df.columns:
            m = df[col].astype(str).str.extract(regex, expand=False)
            y = pd.to_numeric(m, errors='coerce')
            year = year.fillna(y)

    return year.astype('Int64')


def to_decade(year: pd.Series) -> pd.Series:
    return ((year // 10) * 10).astype('Int64')


def ascii_hist(series: pd.Series, width: int = 50, bar_char: str = '█') -> str:
    if series.empty:
        return '(no data)'
    values = series.sort_index()
    maxv = values.max()
    lines = []
    for idx, val in values.items():
        bar_len = int(round((val / maxv) * width)) if maxv else 0
        lines.append(f"{int(idx)}s | {bar_char * bar_len} {val}")
    return "\n".join(lines)


def inline_svg_bar(series: pd.Series, width: int = 600, height: int = 220, pad: int = 32, bar_color: str = '#4C78A8') -> str:
    if series.empty:
        return '<svg width="600" height="60" xmlns="http://www.w3.org/2000/svg"><style>text{fill:#fff}</style><text x="10" y="35">No data</text></svg>'

    values = series.sort_index()
    n = len(values)
    maxv = values.max()
    bw = (width - 2 * pad) / max(n, 1)
    scale = (height - 2 * pad) / maxv if maxv else 1

    svg = [f'<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">']
    svg.append('<style>text{fill:#fff}</style>')
    # axes
    svg.append(f'<line x1="{pad}" y1="{height - pad}" x2="{width - pad}" y2="{height - pad}" stroke="#999"/>')
    svg.append(f'<line x1="{pad}" y1="{pad}" x2="{pad}" y2="{height - pad}" stroke="#999"/>')

    # bars
    for i, (dec, val) in enumerate(values.items()):
        x = pad + i * bw + 2
        bh = val * scale
        y = height - pad - bh
        svg.append(f'<rect x="{x:.1f}" y="{y:.1f}" width="{bw - 4:.1f}" height="{bh:.1f}" fill="{bar_color}"/>')
        svg.append(f'<text x="{x + (bw - 4) / 2:.1f}" y="{height - pad + 14}" font-size="10" text-anchor="middle">{int(dec)}</text>')

    # label
    svg.append(f'<text x="{pad}" y="{pad - 8}" font-size="12">Artists by Decade</text>')
    svg.append('</svg>')
    return "\n".join(svg)


# ----------------------
# Load data & apply filters
# ----------------------
all_path = 'all.csv'
df_all = pd.read_csv(all_path)

# Build exclusion masks (case-insensitive)
genres = df_all.get('Genres', pd.Series('', index=df_all.index)).astype(str)
albums = df_all.get('Album', pd.Series('', index=df_all.index)).astype(str)

genre_mask = genres.str.contains('|'.join(FILTER_GENRE_TERMS), case=False, na=False)
album_mask = albums.str.contains('|'.join(FILTER_ALBUM_TERMS), case=False, na=False)

excluded_mask = genre_mask | album_mask

df_excluded = df_all[excluded_mask].copy()
df_included = df_all[~excluded_mask].copy()

# Clean artists (both included and excluded)
for frame in (df_included, df_excluded):
    frame['Cleaned_Artist'] = clean_artist_col(frame.get('Artist', pd.Series('', index=frame.index)))

# ----------------------
# Derive Year/Decade and aggregate
# ----------------------
df_included['Year'] = infer_year_from(df_included)
df_included['Decade'] = to_decade(df_included['Year'])

# Count artists per decade, after filtering to artists with >1 tracks in that decade
raw = df_included.dropna(subset=['Decade']).groupby(['Decade', 'Cleaned_Artist']).size()
filtered = raw[raw > 1]
artist_decade_counts = filtered.groupby(level=0).size().sort_index()
by_decade = filtered.sort_values(ascending=False).groupby(level=0).head(100)

# ----------------------
# Visualization
# ----------------------
fig, ax = plt.subplots(figsize=(10, 5))
if not artist_decade_counts.empty:
    artist_decade_counts.plot(kind='bar', ax=ax, color='#4C78A8')
ax.set_title('Artists by Decade (>1 tracks per artist, per decade)')
ax.set_xlabel('Decade')
ax.set_ylabel('Count of artists')
ax.grid(axis='y', linestyle='--', alpha=0.3)
fig.tight_layout()
fig.savefig(IMG_PATH, dpi=160)
plt.close(fig)

# ----------------------
# Markdown export (Obsidian-friendly)
# ----------------------
lines = []
lines.append('# Music decades report (refactored)')
lines.append('')

included_tracks = len(df_included)
unique_artists = df_included['Cleaned_Artist'].nunique()
with_year = int(df_included['Year'].notna().sum())
missing_year = int(df_included['Year'].isna().sum())

lines.append('Summary:')
lines.append(f'- Included tracks: {included_tracks:,} (unique artists: {unique_artists:,})')
lines.append(f'- Year coverage: {with_year:,} with year, {missing_year:,} missing')
lines.append(f'- Chart image: {IMG_PATH.as_posix()}')
lines.append('')

lines.append('## Artists by decade (count of artists with >1 tracks, per decade)')
if artist_decade_counts.empty:
    lines.append('_No qualifying artists_')
else:
    lines.append(artist_decade_counts.to_frame('Artists').to_markdown())
lines.append('')

lines.append('### ASCII histogram')
lines.append('')
lines.append('```')
lines.append(ascii_hist(artist_decade_counts))
lines.append('```')
lines.append('')

lines.append('### Inline SVG chart')
lines.append('')
lines.append(inline_svg_bar(artist_decade_counts))
lines.append('')

lines.append('## Top artists per decade (up to 100 each)')
if by_decade.empty:
    lines.append('_No data_')
else:
    for dec in sorted(by_decade.index.get_level_values(0).unique()):
        sub = by_decade.xs(dec, level=0)
        lines.append(f'### {int(dec)}s')
        lines.extend([f'- {artist}: {cnt}' for artist, cnt in sub.items()])
        lines.append('')

# Supplemental: Missing-year artists (within included set)
lines.append('## Supplemental: Artists with tracks excluded due to missing Year')
missing_by_artist = (
    df_included[df_included['Year'].isna()]
    .groupby('Cleaned_Artist')
    .size()
    .sort_values(ascending=False)
)
if missing_by_artist.empty:
    lines.append('_None_')
else:
    top_missing = missing_by_artist.head(50)
    lines.append(top_missing.to_frame('Tracks Missing Year').to_markdown())
    rem = int(missing_by_artist.shape[0] - top_missing.shape[0])
    if rem > 0:
        lines.append(f'_…and {rem} more_')
lines.append('')

# Supplemental: Excluded-by-filters artists (classical/choral/baroque/Bach)
lines.append('## Supplemental: Artists excluded by classical filters')
excluded_counts = df_excluded.groupby('Cleaned_Artist').size().sort_values(ascending=False)
lines.append(f'- Excluded tracks: {len(df_excluded):,}; unique excluded artists: {excluded_counts.index.nunique():,}')
if excluded_counts.empty:
    lines.append('_None_')
else:
    lines.append(excluded_counts.to_frame('Tracks Excluded').to_markdown())

md_refactored = '\n'.join(lines)
display(Markdown(md_refactored))
print(md_refactored)

# Music decades report (refactored)

Summary:
- Included tracks: 1,160 (unique artists: 414)
- Year coverage: 1,036 with year, 124 missing
- Chart image: obsidian_export/artists_by_decade.png

## Artists by decade (count of artists with >1 tracks, per decade)
|   Decade |   Artists |
|---------:|----------:|
|     1950 |         7 |
|     1960 |         9 |
|     1970 |        18 |
|     1980 |         6 |
|     1990 |        21 |
|     2000 |        38 |
|     2010 |        32 |
|     2020 |         8 |

### ASCII histogram

```
1950s | █████████ 7
1960s | ████████████ 9
1970s | ████████████████████████ 18
1980s | ████████ 6
1990s | ████████████████████████████ 21
2000s | ██████████████████████████████████████████████████ 38
2010s | ██████████████████████████████████████████ 32
2020s | ███████████ 8
```

### Inline SVG chart

<svg width="600" height="220" xmlns="http://www.w3.org/2000/svg">
<style>text{fill:#fff}</style>
<line x1="32" y1="188" x2="568" y2="188" stroke="#999"/>
<line x1="32" y1="32" x2="32" y2="188" stroke="#999"/>
<rect x="34.0" y="159.3" width="63.0" height="28.7" fill="#4C78A8"/>
<text x="65.5" y="202" font-size="10" text-anchor="middle">1950</text>
<rect x="101.0" y="151.1" width="63.0" height="36.9" fill="#4C78A8"/>
<text x="132.5" y="202" font-size="10" text-anchor="middle">1960</text>
<rect x="168.0" y="114.1" width="63.0" height="73.9" fill="#4C78A8"/>
<text x="199.5" y="202" font-size="10" text-anchor="middle">1970</text>
<rect x="235.0" y="163.4" width="63.0" height="24.6" fill="#4C78A8"/>
<text x="266.5" y="202" font-size="10" text-anchor="middle">1980</text>
<rect x="302.0" y="101.8" width="63.0" height="86.2" fill="#4C78A8"/>
<text x="333.5" y="202" font-size="10" text-anchor="middle">1990</text>
<rect x="369.0" y="32.0" width="63.0" height="156.0" fill="#4C78A8"/>
<text x="400.5" y="202" font-size="10" text-anchor="middle">2000</text>
<rect x="436.0" y="56.6" width="63.0" height="131.4" fill="#4C78A8"/>
<text x="467.5" y="202" font-size="10" text-anchor="middle">2010</text>
<rect x="503.0" y="155.2" width="63.0" height="32.8" fill="#4C78A8"/>
<text x="534.5" y="202" font-size="10" text-anchor="middle">2020</text>
<text x="32" y="24" font-size="12">Artists by Decade</text>
</svg>

## Top artists per decade (up to 100 each)
### 1950s
- Ella Fitzgerald: 5
- Julie London: 4
- The Chordettes: 3
- The Four Freshmen: 3
- Nat King Cole: 2
- Charlie Parker: 2
- Miles Davis: 2

### 1960s
- The Beatles: 30
- Stan Getz: 5
- The Doors: 4
- Bob Dylan: 3
- Louis Armstrong: 2
- Buffalo Springfield: 2
- Nat King Cole: 2
- Simon & Garfunkel: 2
- The Monkees: 2

### 1970s
- Steely Dan: 9
- Elton John: 7
- Queen: 5
- Billy Joel: 5
- Stevie Wonder: 4
- Electric Light Orchestra: 4
- Earth: 3
- Bruce Springsteen: 3
- Neil Young: 2
- Eagles: 2
- Dire Straits: 2
- Bob Dylan: 2
- Steve Miller Band: 2
- Squeeze: 2
- The Hollies: 2
- The Beatles: 2
- Supertramp: 2
- Bee Gees: 2

### 1980s
- Pixies: 6
- Tears For Fears: 5
- Billy Joel: 3
- Erasure: 2
- Duran Duran: 2
- Michael Jackson: 2

### 1990s
- Elliott Smith: 29
- Green Day: 19
- Buena Vista Social Club: 14
- Weezer: 12
- Radiohead: 8
- Nirvana: 5
- Nas: 4
- The Notorious B.I.G.: 4
- Ms. Lauryn Hill: 4
- Big L: 3
- The Cranberries: 3
- Cocteau Twins: 3
- Benny Golson: 3
- Norbert Kraft: 2
- Jeff Buckley: 2
- Heatmiser: 2
- Smash Mouth: 2
- Depeche Mode: 2
- Sugar Ray: 2
- Adam Holzman: 2
- A Tribe Called Quest: 2

### 2000s
- The Shins: 51
- Elliott Smith: 47
- The Strokes: 30
- Radiohead: 23
- Franz Ferdinand: 22
- Fountains Of Wayne: 9
- Hot Hot Heat: 9
- Green Day: 9
- Muse: 9
- Weezer: 6
- Antonio Lauro: 5
- Wilco: 4
- The White Stripes: 4
- Modest Mouse: 4
- Interpol: 4
- Elefant: 3
- Arctic Monkeys: 3
- The All-American Rejects: 3
- Simple Plan: 3
- The Fratellis: 3
- Good Charlotte: 3
- MGMT: 3
- Streetlight Manifesto: 3
- Britney Spears: 2
- Bobby Darin: 2
- DANGERDOOM: 2
- Blonde Redhead: 2
- Albert Hammond Jr: 2
- Avril Lavigne: 2
- Beyoncé: 2
- John Adams: 2
- Lily Allen: 2
- The New Pornographers: 2
- The Bravery: 2
- Matchbox Twenty: 2
- OK Go: 2
- Pendulum: 2
- Talbot Tagora: 2

### 2010s
- Alvvays: 18
- Pile: 15
- Ariana Grande: 13
- MGMT: 12
- Naomi Punk: 12
- Kendrick Lamar: 7
- Crumb: 5
- Parquet Courts: 4
- Two Door Cinema Club: 4
- Mac DeMarco: 4
- James Elkington: 4
- Ty Segall: 3
- Caroline Shaw: 3
- Dirty Projectors: 3
- Deca: 3
- Foster The People: 3
- Broken Bells: 2
- Big Thief: 2
- Peebs The Prophet: 2
- Open Mike Eagle: 2
- Nathan Salsburg: 2
- Jaeden Camstra: 2
- Girls: 2
- Jonwayne: 2
- Katy Perry: 2
- Freddie Gibbs: 2
- Passion Pit: 2
- Tyler: 2
- Tame Impala: 2
- Sky Ferreira: 2
- The Strokes: 2
- Steve Lacy: 2

### 2020s
- SPIRIT OF THE BEEHIVE: 3
- Brendan Eder Ensemble: 3
- Nicholas Britell: 3
- Sabrina Carpenter: 3
- Taylor Swift: 3
- Adrianne Lenker: 3
- Beyoncé: 3
- Alex G: 2

## Supplemental: Artists with tracks excluded due to missing Year
| Cleaned_Artist           |   Tracks Missing Year |
|:-------------------------|----------------------:|
| Bill Evans               |                    14 |
| Os Tincoãs               |                    12 |
| Kendrick Lamar           |                     7 |
| ABBA                     |                     4 |
| Muse                     |                     4 |
| Electric Light Orchestra |                     3 |
| Billy Joel               |                     3 |
| Stan Getz                |                     3 |
| The Turtles              |                     3 |
| Masayoshi Takanaka       |                     2 |
| Madness                  |                     2 |
| Katy Perry               |                     2 |
| Barry Manilow            |                     2 |
| BOYS LIKE GIRLS          |                     2 |
| America                  |                     2 |
| Bobby Darin              |                     2 |
| Andrew Gold              |                     1 |
| Blue Scholars            |                     1 |
| Antônio Carlos Jobim     |                     1 |
| Chet Baker               |                     1 |
| Count Bass D             |                     1 |
| Carole King              |                     1 |
| Charles Mingus           |                     1 |
| Blur                     |                     1 |
| Blue Öyster Cult         |                     1 |
| Earth                    |                     1 |
| Donovan                  |                     1 |
| Five For Fighting        |                     1 |
| Frank Sinatra            |                     1 |
| Glenn Miller             |                     1 |
| Gorillaz                 |                     1 |
| Fred Astaire             |                     1 |
| Gerry Rafferty           |                     1 |
| Imogen Heap              |                     1 |
| Heart                    |                     1 |
| Daryl Hall & John Oates  |                     1 |
| DEVO                     |                     1 |
| Dean Martin              |                     1 |
| Don McLean               |                     1 |
| Cheap Trick              |                     1 |
| Buffalo Springfield      |                     1 |
| Looking Glass            |                     1 |
| Julie London             |                     1 |
| John Coltrane            |                     1 |
| John Denver              |                     1 |
| James Taylor             |                     1 |
| Jackson Browne           |                     1 |
| Raspberries              |                     1 |
| Modest Mouse             |                     1 |
| Robby Poitevin           |                     1 |
_…and 23 more_

## Supplemental: Artists excluded by classical filters
- Excluded tracks: 1,063; unique excluded artists: 206
| Cleaned_Artist                                    |   Tracks Excluded |
|:--------------------------------------------------|------------------:|
| Johann Sebastian Bach                             |                88 |
| of Montreal                                       |                60 |
| Philip Glass                                      |                45 |
| Frédéric Chopin                                   |                37 |
| Göran Söllscher                                   |                34 |
| Catrin Finch                                      |                32 |
| András Schiff                                     |                31 |
| Wolfgang Amadeus Mozart                           |                29 |
| Julian Rachlin                                    |                27 |
| Augsburger Domsingknaben                          |                24 |
| The Beach Boys                                    |                23 |
| Emerson String Quartet                            |                21 |
| Maurice Ravel                                     |                20 |
| Eduardo Fernandez                                 |                19 |
| Ludwig van Beethoven                              |                17 |
| Vladimir Ashkenazy                                |                17 |
| Víkingur Ólafsson                                 |                15 |
| Arthur Grumiaux                                   |                13 |
| Rosalyn Tureck                                    |                13 |
| Andrés Segovia                                    |                13 |
| Claude Debussy                                    |                12 |
| Nicanor Zabaleta                                  |                12 |
| Mischa Maisky                                     |                11 |
| Christiane Oelze                                  |                11 |
| Andrei Gavrilov                                   |                10 |
| Jean Louis Steuerman                              |                10 |
| Manuel Seco de Arpe                               |                 9 |
| Janine Jansen                                     |                 9 |
| Ramón Paus                                        |                 9 |
| The Sarum Consort                                 |                 8 |
| Henry Purcell                                     |                 8 |
| George Frideric Handel                            |                 8 |
| Choir of Gonville & Caius College                 |                 8 |
| David Lang                                        |                 8 |
| Claudio Arrau                                     |                 7 |
| Joseph Haydn                                      |                 7 |
| Gabriel Fauré                                     |                 6 |
| Franz Schubert                                    |                 6 |
| Wilhelm Kempff                                    |                 6 |
| Severino Gazzelloni                               |                 6 |
| Pepe Romero                                       |                 6 |
| Robert Schumann                                   |                 6 |
| Irena Grafenauer                                  |                 6 |
| Jian Wang                                         |                 6 |
| John C. Williams                                  |                 5 |
| Alexandre Lagoya                                  |                 5 |
| Giuliano Carmignola                               |                 5 |
| Wolfgang Rübsam                                   |                 5 |
| Heinz Holliger                                    |                 5 |
| Johannes Brahms                                   |                 4 |
| Arcangelo Corelli                                 |                 4 |
| Amsterdam Loeki Stardust Quartet                  |                 4 |
| Camille Saint-Saëns                               |                 4 |
| Antonio Vivaldi                                   |                 4 |
| I Musici                                          |                 4 |
| Richard Wagner                                    |                 4 |
| Nelson Freire                                     |                 4 |
| Ophira Zakai                                      |                 4 |
| Maria Graf                                        |                 4 |
| Kaori Muraji                                      |                 4 |
| Anne Müller                                       |                 3 |
| Domenico Scarlatti                                |                 3 |
| Benjamin Grosvenor                                |                 3 |
| Pyotr Ilyich Tchaikovsky                          |                 3 |
| Zoltán Kocsis                                     |                 3 |
| Josquin des Prez                                  |                 3 |
| St. Albans Cathedral Choir                        |                 3 |
| Thomanerchor Leipzig                              |                 3 |
| Vampire Weekend                                   |                 3 |
| The Tallis Scholars                               |                 3 |
| Jean-Philippe Rameau                              |                 3 |
| Janos Sebestyen                                   |                 3 |
| Antonín Dvořák                                    |                 3 |
| Eric Whitacre                                     |                 3 |
| George Gershwin                                   |                 3 |
| Floraleda Sacchi                                  |                 3 |
| Felix Mendelssohn                                 |                 3 |
| Emitt Rhodes                                      |                 3 |
| Rafał Blechacz                                    |                 3 |
| Scott Joplin                                      |                 2 |
| Vocalconsort Berlin                               |                 2 |
| Vicente Lusitano                                  |                 2 |
| Shlomo Mintz                                      |                 2 |
| Nobuko Imai                                       |                 2 |
| Simon Standage                                    |                 2 |
| The Millennium                                    |                 2 |
| Andrew Bird                                       |                 2 |
| Arash Safaian                                     |                 2 |
| Federico Agostini                                 |                 2 |
| Edward Elgar                                      |                 2 |
| Edvard Grieg                                      |                 2 |
| Francis Poulenc                                   |                 2 |
| Giacomo Puccini                                   |                 2 |
| Academy of Ancient Music                          |                 2 |
| Márta Kurtág                                      |                 2 |
| Monica Huggett                                    |                 2 |
| Musica Antiqua Köln                               |                 2 |
| Music Lab Collective                              |                 2 |
| Nicola Hall                                       |                 2 |
| Mari Samuelsen                                    |                 2 |
| Lazar Berman                                      |                 2 |
| Kölner Akademie                                   |                 2 |
| John Adams                                        |                 2 |
| Kun-Woo Paik                                      |                 2 |
| Jean Sibelius                                     |                 2 |
| Hélène Grimaud                                    |                 2 |
| Gustav Mahler                                     |                 2 |
| Henryk Górecki                                    |                 2 |
| Hilary Hahn                                       |                 2 |
| Martha Argerich                                   |                 2 |
| Daniel Hope                                       |                 2 |
| Arvo Pärt                                         |                 2 |
| Beirut                                            |                 2 |
| Daniel Chorzempa                                  |                 2 |
| François Couperin                                 |                 2 |
| Carl Philipp Emanuel Bach                         |                 2 |
| Alan Loveday                                      |                 1 |
| Famous Classics                                   |                 1 |
| Dmitri Shostakovich                               |                 1 |
| David Bell                                        |                 1 |
| David Chalmin                                     |                 1 |
| David Garrett                                     |                 1 |
| Elizabeth Joy Roe                                 |                 1 |
| Cologne Chamber Orchestra                         |                 1 |
| Daniel Barenboim                                  |                 1 |
| Daniel Lozakovich                                 |                 1 |
| Christoph Willibald Gluck                         |                 1 |
| Carducci String Quartet                           |                 1 |
| Carl Maria von Weber                              |                 1 |
| Bengt Forsberg                                    |                 1 |
| Anna Gourari                                      |                 1 |
| Avi Avital                                        |                 1 |
| Anton Arensky                                     |                 1 |
| Arnold Schoenberg                                 |                 1 |
| Craig Ogden                                       |                 1 |
| Carl Nielsen                                      |                 1 |
| Accademia Bizantina                               |                 1 |
| Academy of St. Martin in the Fields Wind Ensemble |                 1 |
| A Winged Victory for the Sullen                   |                 1 |
| Akiko Suwanai                                     |                 1 |
| Alonso Lobo                                       |                 1 |
| Alfred Brendel                                    |                 1 |
| Giaches de Wert                                   |                 1 |
| Gewandhausorchester                               |                 1 |
| Frederick Delius                                  |                 1 |
| Francesco Tristano                                |                 1 |
| Fanny Mendelssohn                                 |                 1 |
| Erik Satie                                        |                 1 |
| Evgeny Kissin                                     |                 1 |
| Julian Lloyd Webber                               |                 1 |
| Johann Strauss I                                  |                 1 |
| Johann Christoph Friedrich Bach                   |                 1 |
| Johann Christian Bach                             |                 1 |
| Joseph Banowetz                                   |                 1 |
| John Cage                                         |                 1 |
| Judee Sill                                        |                 1 |
| Jules Massenet                                    |                 1 |
| Gianluca Cascioli                                 |                 1 |
| Hai-Kyung Suh                                     |                 1 |
| Harry Nilsson                                     |                 1 |
| Giovanni Battista Pergolesi                       |                 1 |
| Gordon Fergus-Thompson                            |                 1 |
| Graham Barber                                     |                 1 |
| Gregorio Allegri                                  |                 1 |
| Grizzly Bear                                      |                 1 |
| Gustav Holst                                      |                 1 |
| Jan Vogler                                        |                 1 |
| Isaac Albéniz                                     |                 1 |
| Igor Stravinsky                                   |                 1 |
| Hörður Áskelsson                                  |                 1 |
| Håkan Hardenberger                                |                 1 |
| Jean Guillou                                      |                 1 |
| Jenő Jandó                                        |                 1 |
| Jeremy Ovenden                                    |                 1 |
| Maurice André                                     |                 1 |
| Marisa Robles                                     |                 1 |
| Love                                              |                 1 |
| Laurence Equilbey                                 |                 1 |
| Konrad Hünteler                                   |                 1 |
| Karl Jenkins                                      |                 1 |
| Lang Lang                                         |                 1 |
| Philip Stopford                                   |                 1 |
| Murray Perahia                                    |                 1 |
| Max Bruch                                         |                 1 |
| Michael Nyman                                     |                 1 |
| Mikalojus Konstantinas Čiurlionis                 |                 1 |
| Niccolò Paganini                                  |                 1 |
| Paul Smith                                        |                 1 |
| Quatuor Mosaïques                                 |                 1 |
| Ola Gjeilo                                        |                 1 |
| Salvatore Accardo                                 |                 1 |
| Roger Eno                                         |                 1 |
| Roberto Michelucci                                |                 1 |
| Rainer Kussmaul                                   |                 1 |
| Steve Reich                                       |                 1 |
| Slovak Philharmonic                               |                 1 |
| Strawberry Alarm Clock                            |                 1 |
| The Decemberists                                  |                 1 |
| The Kinks                                         |                 1 |
| Stuttgart Chamber Orchestra                       |                 1 |
| Sufjan Stevens                                    |                 1 |
| Tomaso Albinoni                                   |                 1 |
| Thomas Morley                                     |                 1 |
| The Zombies                                       |                 1 |
| William Grant Still                               |                 1 |
| William Byrd                                      |                 1 |

# Music decades report (refactored)

Summary:
- Included tracks: 1,160 (unique artists: 414)
- Year coverage: 1,036 with year, 124 missing
- Chart image: obsidian_export/artists_by_decade.png

## Artists by decade (count of artists with >1 tracks, per decade)
|   Decade |   Artists |
|---------:|----------:|
|     1950 |         7 |
|     1960 |         9 |
|     1970 |        18 |
|     1980 |         6 |
|     1990 |        21 |
|     2000 |        38 |
|     2010 |        32 |
|     2020 |         8 |

### ASCII histogram

```
1950s | █████████ 7
1960s | ████████████ 9
1970s | ████████████████████████ 18
1980s | ████████ 6
1990s | ████████████████████████████ 21
2000s | ██████████████████████████████████████████████████ 38
2010s | ██████████████████████████████████████████ 32
2020s | ███████████ 8
```

### Inline SVG chart

<svg width="600" height="220" xmlns="http://www.w3.org/2000/svg">
<style>text{fill:#fff}</style>
<line x1="32" y1="188" x2="568" y2="188" stroke="#999"/>
<line x